In [12]:
!pip install imblearn
import os
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, accuracy_score, log_loss
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV



In [3]:
train_files = ['tile00.csv', 'tile01.csv', 'tile02.csv' ,'tile03.csv']
train = pd.concat([pd.read_csv(f) for f in train_files], ignore_index=True)
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

/tmp/ipykernel_1584/2016119411.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


In [3]:
clf = BalancedBaggingClassifier(estimator=DecisionTreeClassifier(), sampling_strategy = 'not majority', replacement=False, n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.00      0.00      0.00       487
         2.0       0.00      0.00      0.00       279
         3.0       1.00      1.00      1.00         1
         4.0       0.00      0.00      0.00       185
         5.0       0.00      0.00      0.00        65
         6.0       0.00      0.00      0.00        46
         7.0       1.00      0.10      0.18        10

    accuracy                           1.00   3139482
   macro avg       0.37      0.26      0.27   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:


clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)
bag_preds = clf.predict(X_test)

print(classification_report(y_test, bag_preds))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.77      0.31      0.44       487
         2.0       0.81      0.25      0.39       279
         3.0       0.00      0.00      0.00         1
         4.0       0.69      0.26      0.38       185
         5.0       0.72      0.35      0.47        65
         6.0       0.79      0.24      0.37        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.60      0.30      0.38   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(), bootstrap = True, 
                                     n_estimators = 10, max_features = 0.5), param_grid)
clf.fit(X_train, y_train)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:717: FutureWarning: Parameter 'base_estimator' of BaggingClassifier is deprecated in favor of 'estimator'. See BaggingClassifier's docstring for more details.
  estimator = estimator.set_params(**clone(parameters, safe=False))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:717: FutureWarning: Parameter 'base_estimator' of BaggingClassifier is deprecated in favor of 'estimator'. See BaggingClassifier's docstring for more details.
  estimator = estimator.set_params(**clone(parameters, safe=False))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:717: FutureWarning: Parameter 'base_estimator' of BaggingClassifier is deprecated in favor of 'estimator'. See BaggingClassifier's docstring for more details.
  estimator = estimator.set_params(**clone(parameters, safe=False))
/home/codespace/.local/lib/python3.10/site-pac

GridSearchCV(estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         max_features=0.5),
             param_grid={'base_estimator__max_depth': [1, 2, 3, 4, 5],
                         'max_samples': [0.05, 0.1, 0.2, 0.5]})

In [5]:
ft_bag_preds = clf.predict(X_test)

print(classification_report(y_test, ft_bag_preds))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.00      0.00      0.00       487
         2.0       0.00      0.00      0.00       279
         3.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00       185
         5.0       0.00      0.00      0.00        65
         6.0       0.00      0.00      0.00        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.12      0.12      0.12   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 10 datasets

In [13]:
train = pd.read_csv('all_tiles_sample1.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/400162801.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6269307156679838
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     59992
           1       0.92      0.70      0.80      7528
           2       0.94      0.56      0.70      2831
           3       0.98      0.61      0.75       480
           4       0.96      0.62      0.75      1931
           5       0.94      0.59      0.72       604
           6       0.96      0.61      0.74       664
           7       0.90      0.52      0.66       308

    accuracy                           0.93     74338
   macro avg       0.94      0.65      0.76     74338
weighted avg       0.93      0.93      0.92     74338

0.925448626543625


In [14]:
train = pd.read_csv('all_tiles_sample2.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/3660217056.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6081435077794497
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     60004
           1       0.91      0.70      0.79      7531
           2       0.94      0.57      0.71      2829
           3       0.97      0.60      0.74       479
           4       0.96      0.60      0.74      1919
           5       0.95      0.57      0.71       603
           6       0.98      0.58      0.73       662
           7       0.92      0.51      0.66       311

    accuracy                           0.92     74338
   macro avg       0.94      0.64      0.75     74338
weighted avg       0.92      0.92      0.92     74338

0.923699857408055


In [15]:
train = pd.read_csv('all_tiles_sample3.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/2669029991.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.654115336203184
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     59999
           1       0.90      0.69      0.78      7533
           2       0.94      0.55      0.69      2828
           3       0.94      0.56      0.70       479
           4       0.95      0.59      0.73      1925
           5       0.94      0.58      0.72       604
           6       0.97      0.58      0.72       660
           7       0.89      0.51      0.65       310

    accuracy                           0.92     74338
   macro avg       0.93      0.63      0.74     74338
weighted avg       0.92      0.92      0.91     74338

0.9214533616723614


In [16]:
train = pd.read_csv('all_tiles_sample4.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/2761326137.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6406593683501998
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     60000
           1       0.91      0.70      0.79      7532
           2       0.95      0.56      0.70      2830
           3       0.96      0.59      0.73       482
           4       0.96      0.61      0.75      1921
           5       0.95      0.60      0.74       605
           6       0.97      0.59      0.73       659
           7       0.92      0.48      0.63       309

    accuracy                           0.92     74338
   macro avg       0.94      0.64      0.75     74338
weighted avg       0.92      0.92      0.92     74338

0.9239958029540747


In [17]:
train = pd.read_csv('all_tiles_sample5.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/394438633.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6416572212469538
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     60002
           1       0.92      0.69      0.79      7519
           2       0.95      0.56      0.71      2833
           3       0.95      0.58      0.72       480
           4       0.95      0.60      0.74      1926
           5       0.95      0.59      0.73       606
           6       0.96      0.57      0.72       663
           7       0.93      0.46      0.61       309

    accuracy                           0.92     74338
   macro avg       0.94      0.63      0.75     74338
weighted avg       0.92      0.92      0.92     74338

0.9233770077214883


In [18]:
train = pd.read_csv('all_tiles_sample6.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/2398969267.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6163977371426571
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     60002
           1       0.92      0.71      0.80      7526
           2       0.94      0.58      0.72      2831
           3       0.97      0.62      0.76       477
           4       0.96      0.62      0.75      1926
           5       0.95      0.62      0.75       605
           6       0.98      0.60      0.75       661
           7       0.94      0.52      0.67       310

    accuracy                           0.93     74338
   macro avg       0.95      0.66      0.77     74338
weighted avg       0.93      0.93      0.92     74338

0.9269418063439964


In [19]:
train = pd.read_csv('all_tiles_sample7.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/2799844726.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6228144893063741
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     60005
           1       0.91      0.70      0.80      7529
           2       0.94      0.58      0.71      2832
           3       0.94      0.59      0.73       479
           4       0.95      0.62      0.75      1919
           5       0.95      0.61      0.74       606
           6       0.97      0.59      0.73       659
           7       0.94      0.50      0.65       309

    accuracy                           0.93     74338
   macro avg       0.94      0.65      0.76     74338
weighted avg       0.93      0.93      0.92     74338

0.9258790927923808


In [20]:
train = pd.read_csv('all_tiles_sample8.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/3082410217.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6313641714750468
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     59984
           1       0.91      0.70      0.79      7541
           2       0.95      0.57      0.71      2833
           3       0.93      0.57      0.71       477
           4       0.96      0.61      0.75      1925
           5       0.98      0.60      0.74       605
           6       0.97      0.57      0.72       662
           7       0.93      0.50      0.65       311

    accuracy                           0.92     74338
   macro avg       0.94      0.64      0.75     74338
weighted avg       0.92      0.92      0.92     74338

0.9241303236568108


In [21]:
train = pd.read_csv('all_tiles_sample9.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/486512829.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6391882794954958
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     59993
           1       0.91      0.70      0.79      7534
           2       0.95      0.56      0.70      2838
           3       0.95      0.57      0.72       476
           4       0.96      0.61      0.74      1925
           5       0.94      0.58      0.72       603
           6       0.97      0.58      0.73       662
           7       0.92      0.48      0.63       307

    accuracy                           0.92     74338
   macro avg       0.94      0.63      0.75     74338
weighted avg       0.92      0.92      0.92     74338

0.9234308160025828


In [22]:
train = pd.read_csv('all_tiles_sample10.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_16082/2478534943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.6392773057953848
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     60002
           1       0.91      0.70      0.79      7527
           2       0.94      0.57      0.71      2835
           3       0.97      0.59      0.73       479
           4       0.95      0.59      0.73      1922
           5       0.96      0.61      0.74       604
           6       0.96      0.58      0.72       660
           7       0.92      0.48      0.63       309

    accuracy                           0.92     74338
   macro avg       0.94      0.64      0.75     74338
weighted avg       0.92      0.92      0.92     74338

0.9239150905324329
